<a href="https://colab.research.google.com/github/avocadopelvis/mental-load/blob/main/mental_load_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# import libraries
import os
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import Dataset, DataLoader
from torch import nn

In [2]:
# use gpu if available 
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cuda device


In [3]:
# load data
df = pd.read_csv("/content/drive/MyDrive/MENTAL-LOAD-DATASET/final.csv")
df = df.drop(df.columns[0], axis = 1)

# split data into training & testing data
train, test = train_test_split(df, test_size=0.3, random_state = 42)

In [4]:
class SignalDataset(Dataset):
    def __init__(self, df):
      # get x and convert to tensor
      self.x = torch.tensor(df.iloc[:, 0:3840].values)
      # get y and convert to tensor
      self.y = torch.tensor(df["label"].values)
      # get number of samples
      self.n_samples = df.shape[0]

    def __len__(self):
      return self.n_samples

    def __getitem__(self, index):
      return self.x[index], self.y[index]

In [5]:
train_dataset = SignalDataset(train)
test_dataset = SignalDataset(test)

In [6]:
train_dataloader = DataLoader(train_dataset, batch_size=10, shuffle=True) #,num_workers = 2
test_dataloader = DataLoader(test_dataset, batch_size=10, shuffle=True)

In [7]:
# hyperparameters
input_size = 3840
hidden_size = 2500 
num_classes = 3
num_epochs = 20
batch_size = 64
learning_rate = 0.001

In [8]:
# FULLY CONNECTED NEURAL NETWORK

class NeuralNetwork(nn.Module):
  def __init__(self, input_size, hidden_size, num_classes):
    super(NeuralNetwork, self).__init__()
    self.l1 = nn.Linear(input_size, hidden_size)
    self.relu = nn.ReLU()
    self.l2 = nn.Linear(hidden_size, num_classes)

  def forward(self, x):
    out = self.l1(x)
    out = self.relu(out)
    out = self.l2(out)
    return out

model = NeuralNetwork(input_size, hidden_size, num_classes)
print(model)

NeuralNetwork(
  (l1): Linear(in_features=3840, out_features=2500, bias=True)
  (relu): ReLU()
  (l2): Linear(in_features=2500, out_features=3, bias=True)
)


In [9]:
# LOSS & OPTIMIZER
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)
# optimizer = torch.optim.SGD(model.parameters(), lr= learning_rate, momentum=0.9)

In [10]:
# TRAINING LOOP
n_total_steps = len(train_dataloader)

for epoch in range(num_epochs):
  for i, data in enumerate(train_dataloader, 0):
    # get the inputs and labels
    inputs, labels = data

    # forward
    outputs = model(inputs.float())
    loss = criterion(outputs, labels)

    # backward
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (i+1)%100 == 0:
      print(f'epoch {epoch+1} / {num_epochs}, step {i+1} / {n_total_steps}, loss = {loss.item():.4f}')

print("Finished Training")

epoch 1 / 20, step 100 / 147, loss = 1.0976
epoch 2 / 20, step 100 / 147, loss = 1.1976
epoch 3 / 20, step 100 / 147, loss = 0.3637
epoch 4 / 20, step 100 / 147, loss = 0.2122
epoch 5 / 20, step 100 / 147, loss = 0.0198
epoch 6 / 20, step 100 / 147, loss = 0.0205
epoch 7 / 20, step 100 / 147, loss = 0.0107
epoch 8 / 20, step 100 / 147, loss = 0.1463
epoch 9 / 20, step 100 / 147, loss = 0.3452
epoch 10 / 20, step 100 / 147, loss = 0.4578
epoch 11 / 20, step 100 / 147, loss = 0.0000
epoch 12 / 20, step 100 / 147, loss = 0.0164
epoch 13 / 20, step 100 / 147, loss = 0.0034
epoch 14 / 20, step 100 / 147, loss = 0.0202
epoch 15 / 20, step 100 / 147, loss = 0.0017
epoch 16 / 20, step 100 / 147, loss = 0.0003
epoch 17 / 20, step 100 / 147, loss = 0.0003
epoch 18 / 20, step 100 / 147, loss = 0.0003
epoch 19 / 20, step 100 / 147, loss = 0.0005
epoch 20 / 20, step 100 / 147, loss = 0.0005
Finished Training


In [11]:
# TESTING
with torch.no_grad():
  n_correct = 0
  n_samples = 0
  for inputs, labels in test_dataloader:
    inputs = inputs.to(device)
    labels = labels.to(device)
    outputs = model(inputs.float())

    # value, index (We don't need value)
    _, predictions = torch.max(outputs, 1)
    n_samples += labels.shape[0]
    n_correct += (predictions == labels).sum().item()

  acc = 100.0 * n_correct / n_samples
  print(f'accuracy = {acc}')

RuntimeError: ignored

In [26]:
# b = test.iloc[:1]
# b = b.drop(b.columns[3840], axis = 1)
# a = torch.tensor(b.values)
# output = model(a.float())
# output

tensor([[-9.6091, -5.8245,  4.5248]], grad_fn=<AddmmBackward0>)
